<a href="https://colab.research.google.com/github/joe-granick/CS439/blob/main/data_ingest10K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install sec_edgar_downloader

In [1]:
!pip install datamule

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.1/733.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.4/773.4 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 70.4 MB/s eta 0:00:00


In [6]:
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup
import glob, os, re, pprint

# configure downloader

In [7]:
USER  = "Joe Granick (jmgranick@gmail.com)"
EMAIL = "jmgranick@gmail.com"
DIR = "10k_filings"
TICKERS = ["AAPL", "MSFT", "GOOGL"]
FILING_YEARS = 2
FILING_TYPE = "10-K"


dl = Downloader(USER, EMAIL, DIR)
for ticker in TICKERS:
  # creates folder for each filing
  dl.get(FILING_TYPE, ticker,limit=FILING_YEARS)

# Read in Text Corpus From Folder

In [19]:
dir_pattern = os.path.join(DIR,"sec-edgar-filings", "*", FILING_TYPE,"*","full-submission.txt")
filings = {}

for path in glob.glob(dir_pattern):
  folders = path.split(os.sep)
  ticker = folders[-4]
  acc_num = folders[-2]
  xx = acc_num.split("-")[1]
  year = "20" + xx if len(xx) == 2 else xx

  with open(path, "r",encoding="utf-8") as f:
    files=f.read()
  text = BeautifulSoup(files,"lxml").get_text(separator="\n")
  filings.setdefault(ticker,{})[year] = text



In [21]:
# check that downloaded files are properly loaded
print("10k filings", list(filings.keys()))
print("Filings pulled for AAPL:", list(filings["AAPL"].keys()))
print("Filings pulled for MSFT:", list(filings["MSFT"].keys()))
print("Filings pulled for GOOGL:", list(filings["GOOGL"].keys()))

10k filings ['GOOGL', 'AAPL', 'MSFT']
Filings pulled for AAPL: ['2023', '2024']
Filings pulled for MSFT: ['2023', '2024']
Filings pulled for GOOGL: ['2025', '2024']
